In [1]:
import json
for i in [0]:
    data = []
    with open("/Users/philip/Documents/PhD/data/ArenaData/arena_fits/2015-07-05.json") as f:
        data = f.readlines()

    json_lines = []
    
    for line in data:
        jsline = json.loads(line)
        json_lines.append(jsline)

In [51]:
import pandas as pd

frame = pd.DataFrame.from_dict(json_lines)

In [52]:
# rebuild dataframe
# make dataframe of dicts nested in 'value' column
value = pd.DataFrame(list(frame['value']))
del frame['value']

# make dataframe of dicts nested in 'trackeeHistory' column
del value['trackeeHistory']

# make dataframe with a 'coordinates' column
averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
coordinates = pd.DataFrame(list(averagecoordinate['avg']))
averagecoordinate = averagecoordinate.join(coordinates)
del averagecoordinate['avg']
del value['averagecoordinate']

# join dataframes
frame = frame.join(value.join(averagecoordinate))
del frame['regionsNodesIds']
del frame['error']
del frame['type']

In [53]:
frame[:3]

,measurementTimestamp,processingTimestamp,sourceMac,coordinates
0,1436047371831,1436047381094,cae9c0d3-a1ef-44a3-b0a9-d58a482ab54b,"[-92.041205, 12.180177, 0.0]"
1,1436047378235,1436047381097,d7936b86-b468-44a8-aa19-fd86fca3bf8f,"[-77.307555, -66.73669, 0.0]"
2,1436047376517,1436047381099,84cac844-e9c9-41aa-a8a6-22e0e8c4b233,"[-100.96046, -6.853413, 0.0]"


In [6]:
# get most frequent mac address
macs = frame['sourceMac'].value_counts()
macs[:5]

2d1ff804-c499-4163-b90f-003df1e4ec65    78664
85a84e08-c967-41d0-b56b-522574796776    74356
2c019c2d-7e9e-42b1-b3bd-1797c918d4a7    16995
e1deda99-163f-4b68-9ade-e1f05d070bf2     8370
6cab229a-e15d-48b7-ab1e-5ddca3d7e283     7712
Name: sourceMac, dtype: int64

In [54]:
from math import floor, ceil
import numpy as np

width = 80; height = 60

bins = np.zeros((height,width))

normalization = 0

for i in range(300): # len(macs)
    # filter for mac address
    filtered_frame = frame[frame['sourceMac'] == macs.index[i]]
    # extract positions
    locations = filtered_frame['coordinates']
    
    positions1 = pd.DataFrame(locations)
    positions = pd.DataFrame(list(positions1['coordinates']))

    f = lambda x: x/3

    # binned positions, still fractional numbers
    binpos = positions.apply(f)
    
    for j in range(len(binpos) - 1):
        if floor(binpos[0][j]) + 40 >= 0 and floor(binpos[0][j]) + 40 < width and \
            floor(binpos[1][j]) + 30 >= 0 and floor(binpos[1][j]) + 30 < height:
            bins[height/2 + floor(binpos[1][j])][width/2 + floor(binpos[0][j])] += 1
            normalization += 1
            
bins /= normalization
    

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:26: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [55]:
bins.sum()

0.99999999999999989

In [21]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure(figsize=(12,8))

X = np.arange(0,len(bins[1]),1)
Y = np.arange(0,len(bins),1)
Xs, Ys = np.meshgrid(X, Y)

Z = bins

ax = Axes3D(fig)
ax.plot_surface(Xs, Ys, Z, rstride=2, cstride=1, cmap='hsv')#, cmap='hot')
plt.show()

In [56]:
im = plt.imread('/Users/philip/Documents/PhD/data-analysis/escience/arena_sensation.png')

#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
# convenience method:
fig, ax = plt.subplots()

#ax.set_xlim((0,1853)) # 1853
#ax.set_ylim((0,1369)) # 1369

ax.imshow(im)
ax.imshow(bins, aspect='auto', cmap='hsv', alpha=1)

#ax.plot(range(1200))

plt.show()

In [13]:
a=8;b=2
a/=b
a

4.0